In [82]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import math
from keras import backend as K
from random import random as random
from random import randint

# jupyter command - allows plots to show up
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import LSTM
from datetime import datetime


# Creates the Dataset to Train From

The following model will be trained to decipher a quadratic equation given values a, b, and c with a solution.

In [83]:
num_samples = 5000
num_features = 3 # a, b and c
num_target = 2

matrix = np.zeros((num_samples,num_features + num_target))

a_range = [-5, 5]
b_range = [-40, 40]
c_range = [-5, 5]
d_range = [-40, 40]
np.random.seed(1)
def generate_numbers (index, num_samples=num_samples):
    if i < num_samples / 5:
        a = np.random.randint(a_range[0], a_range[1])
        b = np.random.randint(b_range[0], b_range[1])
        c = np.random.randint(c_range[0], c_range[1])
        d = np.random.randint(d_range[0], d_range[1])
    elif i < 2 * num_samples / 5:
        a = 1
        b = np.random.random() * (b_range[1] - b_range[0])
        c = 1
        d = np.random.random() * (d_range[1] - d_range[0])
    else:
        a = np.random.random() * (a_range[1] - a_range[0]) + a_range[0]
        b = np.random.random() * (b_range[1] - b_range[0]) + b_range[0]
        c = np.random.random() * (c_range[1] - c_range[0]) + c_range[0]
        d = np.random.random() * (d_range[1] - d_range[0]) + d_range[0]
    return a, b, c, d

for i in range(num_samples):
    a, b, c, d = generate_numbers(i)
    # b**2 - 4ac
    root = (1.0 * a * d + 1.0 * b * c) ** 2 - 4.0 * a * c * b * d
    while abs(a) < 1 or abs(b) < 1 or abs(c) < 1 or abs(d) < 1 or (root < 0):
        a, b, c, d = generate_numbers(i)
    matrix[i, 0] = 1.0 * a * c
    matrix[i, 1] = 1.0 * a * d + 1.0 * b * c
    matrix[i, 2] = 1.0 * b * d
    matrix[i, 3] = -1.0 * b / a
    matrix[i, 4] = -1.0 * d / c

trainX = matrix[:, :num_features]
trainY = matrix[:, num_features:]
trainX = trainX.reshape(num_samples, 1, num_features)
trainY = trainY.reshape(num_samples, 1, num_target)


# Custom Grid Search Function

In [122]:
def custom_grid_search(trainX, trainY, create_model_function, params, batch_size=25, epochs=10, verbose=3, seed=None):
    # creates an array of the keys
    keys = np.array(params.keys())
    # feeds the keys, the full dictionary, and a fresh dictionary to the recursive function
    # to get an array of dictionaries of every possible combination of parameters
    indiv_params = combine_parameters (keys, params, dict())
    # An array which will keep track of the accuracy of each iteration of the parameters
    results = np.zeros(indiv_params.size)
    # Runs each of the possible combinations
    for i in range(indiv_params.size):
        # prevents changing the original set of parameters
        indiv_param = indiv_params[i].copy()
        # Saves batch_size and epochs for the fitting of the model, not the creation/compilation of
        if 'batch_size' in keys:
            batch_size = indiv_param.pop('batch_size', None)
        if 'epochs' in keys:
            epochs = indiv_param.pop('epochs', None)
        # if a seed has been specified, use it
        if seed != None:
            np.random.seed(seed)
        # Creates the model and trains it
        model = create_model(**indiv_param)
        model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
        # Stores this iteration's results
        results[i] = model.evaluate(trainX, trainY)
        # Clears the backend session in an attempt to save memory with each model
        K.clear_session()
    # finds the index of the result with the lowest loss value
    best_i = np.argwhere(results == min(results) )
    # Prints the best score and the parameters that achieved that score
    print results[best_i], indiv_params[best_i]
    # Returns all the results and their respective parameters in order
    return dict(results=results,
                parameters=indiv_params)

'''
  Given a Dictionary, the keys yet to be looped through, and a specific instance of the dictionary 
  it returns an array of dictionaries showing all unique combinations of the values
  
  e.g. given a dictionary (a=[0, 1], b=[0, 1]), it will return the following array
  array([
    dict(a=0, b=0),
    dict(a=0, b=1),
    dict(a=1, b=0),
    dict(a=1, b-1)
  ])
  
  The Structure is as follows:

    The function is given an array of keys ['a', 'b'] and a dictionary (a=[0,1], b=[0, 1])

    It loops through the first key, in this case a ∈ {0, 1} and removes the first key from the array,
    its new value being ['b']

    It then loops through all the values of the first key, creating a variable called specific_dict, 
    which is a dictionary with the given value of a (as well as any previously defined keys in specific_dict)
      (a=0), and (a=1)

    It then calls itself, providing the updated list of keys yet to be iterated through, the full dictionary,
    and the specific_dict each for loop

    Once the list of keys yet to be iterated through is of size 1, it ceases the recursion
  
'''
def combine_parameters(keys, full_dict, specific_dict):
    result = np.array([])
    # If the size of the keys array is 1, that means this is the final key to be looped through, cease recursion
    if keys.size == 1:
        for i in range(len(full_dict[keys[0]])):
            # the result will be the specific dict followed by every value in this 
            specific_dict[keys[0]] = full_dict[keys[0]][i]
            result = np.append(result, specific_dict)
        return result
    else:
        # keeps the first key
        current_key = keys[0]
        # removes the first key from the list of keys that havent been looped through yet
        keys = keys[1:]
        # for every value of the current_key
        for i in range(len(full_dict[current_key])):
            specific_dict = specific_dict.copy()
            specific_dict[current_key] = full_dict[current_key][i]
            result = np.append(result, combine_parameters(keys, full_dict, specific_dict))
    return result

# Example of Implementing the Custom Grid Search Function

In [123]:
def create_model(num_layers=1, num_neurons=64):
    # Parameters Yet to be Tested
    activation_function = 'tanh'
    optimizer = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    # The Tested Parameters
    #num_layers = 6
    #num_neurons = 128*3
    # Creates the structure of the model
    inputs = Input(shape=(1,num_features))
    x = Dense(num_neurons, activation=activation_function)(inputs)
    for i in range(num_layers-1):
        x = Dense(num_neurons, activation=activation_function)(x)
    predictions = Dense(2)(x)
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
# The parameters to be looped through
num_layers = [4, 5, 6, 7]
num_neurons = [64, 128, 256, 128*3, 518]
# The dictionary containing the range of parameters
params = dict(num_layers=num_layers,
              num_neurons=num_neurons)
# The results
results = custom_grid_search(trainX, trainY, create_model_function=create_model, params=params, seed=1)

K.clear_session()

loss = results['results']
params = results['parameters']

for l, p in zip(loss, params):
    print l, p

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
4672/5000 [===========================>..] - ETA: 0sEpoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
4896/5000 [============================>.] - ETA: 0sEpoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
4928/5000 [============================>.] - ETA: 0sEpoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
4224/5000 [========================>.....] - ETA: 0sEpoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
4416/5000 [=========================>....] - ETA: 0sEpoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
4448/5000 [=========================>....] - ETA: 0sEpoch 1/10
Epoch 2/10


KeyboardInterrupt: 

In [108]:
K.clear_session()